In [ ]:
# for child and direct matches

In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
import re

In [ ]:
def score_calculator(Dbase_df,Ass_df,start_list):
    dict_df_80_90 = {}
    dict_df_90_100 = {}
    for i in start_list:
        db =Dbase_df.query('Start_Letter ==@i')
        print(i)
        Ass = Ass_df.query('Start_Letter ==@i')
        combo_df = pd.merge(Ass,db, on ='Start_Letter')
        combo_df['Score']=combo_df.apply(lambda x:fuzz.token_set_ratio(x.processed_name,x.DB_processed_name),axis=1)
        dict_df_80_90[i] = combo_df.query('80 < Score <= 90')
        dict_df_90_100[i] = combo_df.query('Score > 90')
    df_90_100 = pd.concat([pd.concat([v]) for k,v in dict_df_90_100.items()])
    df_80_90 = pd.concat([pd.concat([v]) for k,v in dict_df_80_90.items()])
    return df_90_100,df_80_90

In [ ]:
def score_calculator_v1(Dbase_df,Ass_df,start_list):
    dict_df_others = {}
    dict_df_95_100 = {}
    for i in start_list:
        db =Dbase_df.query('Start_Letter ==@i')
        print(i)
        Ass = Ass_df.query('Start_Letter ==@i')
        combo_df = pd.merge(Ass,db, on ='Start_Letter')
        combo_df['Score']=combo_df.apply(lambda x:fuzz.token_set_ratio(x.processed_name,x.DB_processed_name),axis=1)
        dict_df_95_100[i] = combo_df.query('Score >= 95')
        dict_df_others[i] = combo_df.query('90 >=Score < 95')
        
    df_95_100 = pd.concat([pd.concat([v]) for k,v in dict_df_95_100.items()])
    df_others = pd.concat([pd.concat([v]) for k,v in dict_df_others.items()])
    return df_95_100,df_others

In [2]:
#For the direct matches
Ass_EMEA_focus = pd.read_csv('Focus8k_ind.csv')
Dbase_direct = pd.read_csv('mn_db_names_direct.csv')

Dbase_direct['DB_processed_name'] = Dbase_direct.demandbase_company_name.str.lower()
Dbase_direct['length'] = Dbase_direct.DB_processed_name.str.len()
Dbase_direct = Dbase_direct[Dbase_direct.length > 1][['demandbase_sid','demandbase_company_name','DB_processed_name']]

Ass_EMEA_focus['processed_name'] = Ass_EMEA_focus.sub_name.str.lower()

In [3]:
Dbase_direct['Start_Letter'] = Dbase_direct.DB_processed_name.str[0].str.lower()
Ass_EMEA_focus['Start_Letter'] = Ass_EMEA_focus.processed_name.str[0].str.lower()
start_list_Ass = Ass_EMEA_focus.Start_Letter.unique()
print ("Starting Letters in Account Segment Summary\n",start_list_Ass)
Dbase_direct = Dbase_direct.loc[Dbase_direct['Start_Letter'].isin(start_list_Ass)]
start_list_db = Dbase_direct.Start_Letter.unique()
print ("Starting Letters in Demand Base\n",start_list_db)


Starting Letters in Account Segment Summary
 ['a' 's' 'r' 'j' 'l' 'f' 'd' 'c' 'b' 't' 'n' 'k' 'g' 'e' 'x' 'p' 'i' 'o'
 'y' 'm' 'h' 'z' 'w' 'v' 'q' 'u' 'é' '3' '2' '1' '4' '@' '�' '5' '6' '9'
 '8']
Starting Letters in Demand Base
 ['m' 'b' 'a' 'c' 't' 'l' 'i' 'p' 'r' 'g' 'u' 's' 'h' 'f' 'w' 'n' 'z' 'j'
 'd' 'e' 'o' 'v' 'k' 'q' 'x' 'y' '1' '3' '2' '4' '8' '5' 'é' '9' '@' '6']


In [ ]:
dict_df_100 = {}
for i in start_list_db:
        db =Dbase_direct.query('Start_Letter ==@i')
        print(i)
        Ass = Ass_EMEA_focus.query('Start_Letter ==@i')
        combo_df = pd.merge(Ass,db, on ='Start_Letter')
        combo_df['Score'] = np.where(combo_df.processed_name == combo_df.DB_processed_name ,100,0)
        dict_df_100[i] = combo_df.query('Score == 100')
df_100 = pd.concat([pd.concat([v]) for k,v in dict_df_100.items()])

In [ ]:
df_100.to_csv('Output/df_100_direct_matches.csv')

In [4]:
#Read the direct matches
df_100 = pd.read_csv('Output/df_100_direct_matches.csv')
df_100_dmd_sids = np.array(df_100.demandbase_sid.unique())
df_child_input =Dbase_direct [~Dbase_direct['demandbase_sid'].isin(df_100_dmd_sids)]

In [5]:
#Loading the child keys file
Ass_Focus_Terr_Child = pd.read_csv('Child_de_fr_uk.csv')
Ass_Focus_Terr_Child['processed_name'] = Ass_Focus_Terr_Child.child_name.str.lower()
Ass_Focus_Terr_Child = Ass_Focus_Terr_Child[['sub_std_name_key','child_name','child_key','processed_name']]
Ass_Focus_Terr_Child['Start_Letter'] = Ass_Focus_Terr_Child.processed_name.str[0].str.lower()
Ass_Focus_Terr_Child = Ass_Focus_Terr_Child.sort_values(['Start_Letter'],ascending = True)
df_child_start_list = np.array(df_child_input.Start_Letter.unique())
Ass_Focus_Terr_Child = Ass_Focus_Terr_Child.loc[Ass_Focus_Terr_Child['Start_Letter'].isin(df_child_start_list)]

In [ ]:
dict_df_100_child = {}
for i in df_child_start_list:
        db =df_child_input.query('Start_Letter ==@i')
        print(i)
        Ass = Ass_Focus_Terr_Child.query('Start_Letter ==@i')
        combo_df = pd.merge(Ass,db, on ='Start_Letter')
        combo_df['Score'] = np.where(combo_df.processed_name == combo_df.DB_processed_name ,100,0)
        dict_df_100_child[i] = combo_df.query('Score == 100')
df_100_child = pd.concat([pd.concat([v]) for k,v in dict_df_100_child.items()])

m
b


In [ ]:
len(df_100_child)

### DONT RUN THE BELOW 

In [ ]:
Ass_EMEA_focus = pd.read_csv('Focus8k_ind.csv')
Dbase = pd.read_csv('mn_db_names_1.csv')
pre_mapped_dbs = pd.read_excel('Output/showcase.xlsx')

In [ ]:
pre_mapped_dbs_list = np.array(pre_mapped_dbs.demandbase_sid.unique())
Dbase = Dbase.loc[~Dbase['demandbase_sid'].isin(pre_mapped_dbs_list)]
Dbase= Dbase.drop_duplicates()

In [ ]:
['processed_nAss_EMEA_focusame'] = Ass_EMEA_focus.sub_name.str.replace('\W',' ')
Ass_EMEA_focus['processed_name'].replace(to_replace="[^\x00-\x7F]+", value=r" ", regex=True, inplace=True)
Ass_EMEA_focus ['processed_name']=Ass_EMEA_focus.processed_name.str.strip()
Ass_EMEA_focus['length'] = Ass_EMEA_focus.processed_name.str.len()
Ass_EMEA_focus = Ass_EMEA_focus[Ass_EMEA_focus.length > 1][['sub_std_name_key','sub_name','processed_name','tap_industry']]

Dbase['DB_processed_name']= Dbase.demandbase_company_name.str.replace('\W',' ')
Dbase['DB_processed_name'].replace(to_replace="[^\x00-\x7F]+", value=r" ", regex=True, inplace=True)
Dbase ['DB_processed_name']=Dbase.DB_processed_name.str.strip()
Dbase['length'] = Dbase.DB_processed_name.str.len()
Dbase = Dbase[Dbase.length > 1][['demandbase_sid','demandbase_company_name','DB_processed_name']]

Dbase['Start_Letter'] = Dbase.DB_processed_name.str[0].str.lower()
Ass_EMEA_focus['Start_Letter'] = Ass_EMEA_focus.processed_name.str[0].str.lower()
Dbase =Dbase.sort_values(['Start_Letter'],ascending = True)
Ass_EMEA_focus = Ass_EMEA_focus.sort_values(['Start_Letter'],ascending = True)

In [ ]:
start_list_Ass = Ass_EMEA_focus.Start_Letter.unique()
print ("Starting Letters in Account Segment Summary\n",start_list_Ass)
Dbase = Dbase.loc[Dbase['Start_Letter'].isin(start_list_Ass)]
start_list_db = Dbase.Start_Letter.unique()
print ("Starting Letters in Demand Base\n",start_list_db)

In [ ]:
Start_count_grouped = Dbase.groupby(['Start_Letter'])['DB_processed_name'].count().reset_index()

In [ ]:
#Calling the scoring function and writing the output to pickle files
df_stage1_90_100,df_stage1_80_90= score_calculator(Dbase,Ass_EMEA_focus,start_list_db)

In [ ]:
df_stage1_90_100.to_pickle('Output/df_stage1_90_100.pkl')
df_stage1_80_90.to_pickle('Output/df_stage1_80_90.pkl')


In [ ]:
df_stage1_90_100.to_excel('Output/df_stage1_90_100.xls')
df_stage1_80_90.to_excel('Output/df_stage1_80_90.xls')

In [ ]:
len(df_stage1_95_100)

### Read the Processed File 

In [ ]:
df_stage1_90_100 = pd.read_excel('Output/df_stage1_90_100.xls')
df_stage1_80_90 = pd.read_excel('Output/df_stage1_80_90.xls')
dict_1 = pd.read_csv('dictionary.csv')
frames = [df_stage1_90_100,df_stage1_80_90]
working_df = pd.concat(frames)

In [ ]:
working_df.processed_name =working_df.processed_name+' '
working_df.DB_processed_name =working_df.DB_processed_name+' '
removal_list =list(dict_1.dictionary)
for idx,val in enumerate(removal_list):
    pat1= '|'.join([val])
    working_df.processed_name =working_df.processed_name.str.lower().replace(pat1,' , ',regex= True)
    working_df.DB_processed_name =working_df.DB_processed_name.str.lower().replace(pat1,' , ',regex= True)
working_df['dict_Score']=working_df.apply(lambda x:fuzz.token_set_ratio(x.processed_name,x.DB_processed_name),axis=1)

In [ ]:
working_df.to_excel('Output/df_stage2_All_mn.xls')
df_Stage2_abridged_90_100 = working_df.query('dict_Score >=90')
df_Stage2_abridged_90_100.to_excel('Output/df_Stage2_abridged_90_100_mn.xls')

In [ ]:
df_Stage2_abridged_90_100 = pd.read_excel('Output/df_Stage2_abridged_90_100_mn.xls')
AEC_Stage_1_Final = pd.read_excel('AEC_Stage_1_Final.xlsx')
dbase_list_AEC = AEC_Stage_1_Final.demandbase_sid.unique()
df_Stage2_abridged_90_100_deduped =df_Stage2_abridged_90_100 [~df_Stage2_abridged_90_100['demandbase_sid'].isin(dbase_list_AEC)]
df_Stage2_abridged_90_100_deduped.to_excel('Output/Final.xlsx')

#### WORK STOPPED HERE--

In [ ]:
df_Stage2_90_100.sub_std_name_key.nunique()

In [ ]:
df_stage1_85_95 = pd.read_pickle('Output/df_stage1_85_95.pkl')
df_stage1_75_85 = pd.read_pickle('Output/df_stage1_75_85.pkl')

In [ ]:
df_stage1_matched = df_stage1_95_100_flagged[df_stage1_95_100_flagged.Match == 1]
df_stage1_unmatched = df_stage1_95_100_flagged[df_stage1_95_100_flagged.Match == 0]

In [ ]:
#Combine Unmatched records and the records with scores 85-95
frames1 = [df_stage1_unmatched[['sub_std_name_key', 'sub_name', 'processed_name', 'tap_industry',
       'Start_Letter', 'demandbase_sid', 'demandbase_company_name',
       'DB_processed_name']],df_stage1_85_95[['sub_std_name_key', 'sub_name', 'processed_name', 'tap_industry',
       'Start_Letter', 'demandbase_sid', 'demandbase_company_name',
       'DB_processed_name']]]
df_stage2_input = pd.concat(frames1)

In [ ]:
#Remove matched demandbase_sid's from all further processing
matched_db_list = np.array(df_stage1_matched.demandbase_sid.unique())
df_stage2_input = df_stage2_input.loc[~df_stage2_input['demandbase_sid'].isin(matched_db_list)]

In [ ]:
working_df = df_stage2_input[['demandbase_sid','demandbase_company_name','Start_Letter','DB_processed_name']].drop_duplicates()

In [ ]:
#Loading the child keys file
Ass_Focus_Terr_Child = pd.read_csv('Child_de_fr_uk.csv')
Ass_Focus_Terr_Child['processed_name'] = Ass_Focus_Terr_Child.child_name.str.replace('\W',' ')
Ass_Focus_Terr_Child['processed_name'].replace(to_replace="[^\x00-\x7F]+", value=r" ", regex=True, inplace=True)
Ass_Focus_Terr_Child ['processed_name']=Ass_Focus_Terr_Child.processed_name.str.strip()
Ass_Focus_Terr_Child['length'] = Ass_Focus_Terr_Child.processed_name.str.len()
Ass_Focus_Terr_Child = Ass_Focus_Terr_Child[Ass_Focus_Terr_Child.length > 1][['sub_std_name_key','child_name','child_key','processed_name']]
Ass_Focus_Terr_Child['Start_Letter'] = Ass_Focus_Terr_Child.processed_name.str[0].str.lower()
Ass_Focus_Terr_Child = Ass_Focus_Terr_Child.sort_values(['Start_Letter'],ascending = True)
working_df_Start_list = np.array(working_df.Start_Letter.unique())
Ass_Focus_Terr_Child = Ass_Focus_Terr_Child.loc[Ass_Focus_Terr_Child['Start_Letter'].isin(working_df_Start_list)]

In [ ]:
Ass_Focus_Terr_Child.columns

In [ ]:
df_stage2_95_100,df_stage2_90_95 = score_calculator_v1(working_df,Ass_Focus_Terr_Child,working_df_Start_list)

In [ ]:
df_stage2_95_100.to_pickle('Output/df_stage2_95_100.pkl')
df_stage2_90_95.to_pickle('Output/df_stage2_90_95.pkl')
df_stage2_95_100[0:456].to_excel('Output/df_stage2_95_100_1.xls')
df_stage2_95_100[457:].to_excel('Output/df_stage2_95_100_2.xls')

In [ ]:
df_stage2_95_100.head()